<img src="img\cover.png">

# LEZIONE 3


6 Giugno 2018

<img src="img/xkcd.jpg">

<img src="img/devprod.png">

## RICAPITOLANDO.....

### Sviluppo algoritmo IA:

E' una applicazione del **metodo scientifico** all'informatica!

Si itera il work-flow:

<img src="img/workflow.jpeg">

1. Get data: Pandas 
2. Clean, Prepare & Manipulate Data: Pandas (+ Numpy)
3. Train Model: Scikit-Learn
4. Test Data: Scikit-Learn
5. Improve: Human!

Python è la colla che unisce tutti questi aspetti.

Ripassiamo velocemente Scikit-Learn:

http://scikit-learn.org/stable/tutorial/basic/tutorial.html

E ora ripassiamo con un esempio:

riconoscere le cifre numeriche scritte a mano!

## Esercitazione: un classificatore per prevedere lo stipendio...

Costruiremo un classificatore per prevedere lo stipendio di una persona sulla base di 14 attributi....

L'obiettivo sarà prevedere se lo stipendio sarà minore o maggiore di 50000$ annui (classificazione binaria).

Useremo i dati di censimento US disponibili alla pagina https://archive.ics.uci.edu/ml/datasets/Census+Income salvati sul file income_data_with_header.txt.

I dati sono sia numerici che stringhe...


### Prima però vediamo cosa sono i LabelEncoder

Nel caso di dati non numerici che assumono un valore tra un set di valori finito, occorre "mappare" i valori su numeri.

Per questo scikit-learn mette a disposizioni la classe LabelEncoder...

In [13]:
from sklearn.preprocessing import LabelEncoder
le = preprocessing.LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
print list(le.classes_)
le.transform(["tokyo", "tokyo", "paris"]) 
print list(le.inverse_transform([2, 2, 1]))

['amsterdam', 'paris', 'tokyo']
['tokyo', 'tokyo', 'paris']


Si possono unire fit e transform:

In [15]:
le.fit_transform(["paris", "paris", "tokyo", "amsterdam"])

array([1, 1, 2, 0])

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [17]:
# Input file containing data
input_file = 'income_data_with_header.txt.txt'

In [18]:
# Read the data
data = pd.read_csv('income_data_with_header.txt', index_col=0)
data.head()

,age,workclass,finalweight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [19]:
data.dtypes.tolist()

[dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('O')]

In [20]:
data['workclass'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay', ' Never-worked'], dtype=object)

In [21]:
label_encoders = {}
data_encoded = pd.DataFrame(data.copy())
for col, coltype in zip(data_encoded.columns,data_encoded.dtypes.tolist()):
    if coltype=='object':
        lb = LabelEncoder()
        data_encoded[col] = lb.fit_transform(data_encoded[col])
        label_encoders[col] = lb

data_encoded.head()

,age,workclass,finalweight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [22]:
label_encoders

{'education': LabelEncoder(),
 'marital-status': LabelEncoder(),
 'native-country': LabelEncoder(),
 'occupation': LabelEncoder(),
 'race': LabelEncoder(),
 'relationship': LabelEncoder(),
 'sex': LabelEncoder(),
 'target': LabelEncoder(),
 'workclass': LabelEncoder()}

In [23]:
data_encoded.tail(10)

,age,workclass,finalweight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
32551,32,4,34066,0,6,2,6,0,0,1,0,0,40,39,0
32552,43,4,84661,8,11,2,12,0,4,1,0,0,45,39,0
32553,32,4,116138,12,14,4,13,1,1,1,0,0,11,36,0
32554,53,4,321865,12,14,2,4,0,4,1,0,0,40,39,1
32555,22,4,310152,15,10,4,11,1,4,1,0,0,40,39,0
32556,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32557,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32558,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0
32559,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0
32560,52,5,287927,11,9,2,4,5,4,0,15024,0,40,39,1


In [24]:
len(data_encoded)

32561

In [27]:
col_names = data.columns

In [28]:
X = data_encoded[col_names[:-1]].values
y = data_encoded['target'].values

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
max_depth=5
n_estimators=200
classifier = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [31]:
classifier.score(X_test, y_test)

0.85045598362181274

In [32]:
features_importance = pd.DataFrame(index = col_names[:-1],
                                   data={'Importance':classifier.feature_importances_})
features_importance.sort_values('Importance',ascending=False)

,Importance
relationship,0.235571
capital-gain,0.222163
marital-status,0.169044
education-num,0.155757
age,0.067897
hours-per-week,0.040302
capital-loss,0.038673
education,0.030034
sex,0.021912
occupation,0.013184


## Raspberry PI

Spediamo due parole su questo dispositivo:  https://www.raspberrypi.org/

Hardware: https://www.raspberrypi.org/products/

Documentazione: https://www.raspberrypi.org/documentation/

Progetti: https://projects.raspberrypi.org/en/

Per iniziare: https://projects.raspberrypi.org/en/projects/raspberry-pi-getting-started